# Защита персональных данных клиентов

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

### Описание данных.

**Признаки:** пол, возраст и зарплата застрахованного, количество членов его семьи.  
**Целевой признак:** количество страховых выплат клиенту за последние 5 лет.

## 1. Загрузите и изучите данные.

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from math import sqrt

RANDOM_STATE = 12345

Подключил библиотеки.

In [2]:
data = pd.read_csv('/datasets/insurance.csv')

Загрузил данные.

In [3]:
data.head(5)

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Просмотрел первые 5 значений в таблице.

In [4]:
print(data.columns)

Index(['Пол', 'Возраст', 'Зарплата', 'Члены семьи', 'Страховые выплаты'], dtype='object')


Просмотрел перечень стоблцов.

In [5]:
data.duplicated().sum()

153

Проверил количество явных дубликатов, обнаружил 153 дубликата.

In [6]:
data = data.drop_duplicates()

Удалил дубликаты.

In [7]:
data.duplicated().sum()

0

Убедился в отсутствии дубликатов.

In [8]:
data.isna().sum()

Пол                  0
Возраст              0
Зарплата             0
Члены семьи          0
Страховые выплаты    0
dtype: int64

Проверил количество пропущенных значений.

In [9]:
data.nunique()

Пол                    2
Возраст               46
Зарплата             524
Члены семьи            7
Страховые выплаты      6
dtype: int64

Проверил количество уникальных значений.

In [10]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                4847 non-null   int64  
 1   Возраст            4847 non-null   float64
 2   Зарплата           4847 non-null   float64
 3   Члены семьи        4847 non-null   int64  
 4   Страховые выплаты  4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 227.2 KB


Просмотрел общую информацию по датафрейму, увидел, что столбцы "Возраст" и "Зарплата" имеют формат float.

In [11]:
data['Возраст'] = data['Возраст'].astype(int)
data['Зарплата'] = data['Зарплата'].astype(int)
data.dtypes

Пол                  int64
Возраст              int32
Зарплата             int32
Члены семьи          int64
Страховые выплаты    int64
dtype: object

Изменил тип столбцов "Возраст" и "Зарплата" на int.

In [12]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,4847.000000,4847.000000,4847.000000,4847.000000,4847.000000
mean,0.498453,31.023932,39895.811223,1.203425,0.152259
std,0.500049,8.487995,9972.952441,1.098664,0.468934
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33200.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Проанализировал датафрэим функцией describe().

### Вывод.

Завершил предобработку данных, перехожу к  выполнению проекта.

## 2. Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^TX)^{-1}X^Ty
$$

**Ответ: - не изменится**

**Обоснование:**

Новая матрица признаков $X1$ будет равняться произведению матрицы $X$ на обратимую маьтрицу $P$.

$$
X_{1}=XP
$$

Подставим значение $XP$ в формулу $w$ и вычислим $w_{1}$.

$$
w_{1}=((XP)^TXP)^{-1} (XP)^T y
$$

Расскроем произведение $(XP)^T$:

$$
w_{1}=(P^TX^TXP)^{-1} P^TX^T y
$$

Выполним перегруппировку.

$$
w_{1}=(P^T(X^TX)P)^{-1} P^TX^T y
$$

Расскроем скобки $(P^T(X^TX)P)^{-1}$

$$
w_{1}=P^{-1}(X^TX)^{-1} (P^T)^{-1}P^TX^{T} y
$$

По условию $P$ это обратимая матрица, т.е. произведение $(P^{T})^{-1}P^{T}=E$ (единичная матрица).

$$(P^{T})^{-1}P^{T}=E$$

$$
w_{1}=P^{-1}(X^{T}X)^{-1}EX^{T}y=P^{-1}(X^TX)^{-1} X^T y
$$

Получилась формула для $w$

$$
w_{1}=P^{-1}(X^{T}X)^{-1}X^{T}y=P^{-1}w
$$

Получившееся значение весов $w_{1}$ подставим в формулу для предстазания линейной регрессии.

$$
a=Xw
$$

Подставим в эту формулу значения $X1$ и $w_{1}$

$$
a_{1}=X_{1}w_{1}=XPP^{-1}w
$$

Из условия матрица является обратимой, по этому ее произведение снова равно:

$$
PP^{-1}=E
$$

Подставим значение $PP^{-1}$ в формулу расчета $a1$.

$$
a_{1}=XPP^{-1}w=XEw=Xw=a
$$

### Вывод.

В результате проведенного исследования, мы увидели, что предсказания для матрицы признаков умноженных на
обратимую матрицу равны предсказаниям.

Качество линейной регрессии при умножении признаков на обратимую матрицу не меняется.  

Параметры линейной регрессии в исходной задаче и в преобразованной связаны как:

$$
w_{1}=P^{-1}w
$$

## 3. Алгоритм преобразования

- С помощью библиотери np создадим произвольную квадратную матрицу с размерностью 4х4 (равной количеству столбцов матрицы признаков).
- Матрица обязательно должна быть обратимой, если это не так, то надо создать новую матрицу.
- Перемножим матрицу признаков и полученную обратимую матрицу, тем самым зашифровывая истинные значения.

**Обоснование**
Проведя исследование в п.2 мы сделали вывод, что качество линейной регрессии при умножении признаков на обратимую матрицу не меняется.

## 4. Проверка алгоритма

In [13]:
target = data['Страховые выплаты']
features = data.drop(['Страховые выплаты'] , axis=1)

Выделил признаки.

In [14]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.25, random_state=RANDOM_STATE)

Разделил датафрэим на обучающую и тестовую выборку в размере 75% и 25%.

In [15]:
print(features_train.shape)
print(target_train.shape)
print(features_test.shape)
print(target_test.shape)

(3635, 4)
(3635,)
(1212, 4)
(1212,)


Просмотрел информацию о количестве наблюдений и признаков в каждой таблице.

In [16]:
matrix = np.random.randint(10,size=(4, 4))
print(matrix)

[[9 9 6 4]
 [1 5 1 7]
 [0 9 8 5]
 [0 0 4 5]]


С помощью np создал произвольную матрицу 4х4.

In [17]:
matrix_obr = np.linalg.inv(matrix)
print(matrix_obr)

[[ 0.11332595 -0.01993355 -0.10225175  0.03949797]
 [-0.0073828   0.06644518  0.08157992 -0.16869694]
 [ 0.0166113  -0.14950166  0.06644518  0.12956811]
 [-0.01328904  0.11960133 -0.05315615  0.09634551]]


Проверил матрицу на обратимость.

In [18]:
features_train_trans = features_train.dot(matrix)
features_test_trans = features_test.dot(matrix)

Умножил признаки матриц на сгенерированную матрицу.

In [19]:
model = LinearRegression()
model.fit(features_train, target_train)
predictions = model.predict(features_test)
print('R2', r2_score(target_test, predictions))

R2 0.42307727615837243


Обучил модель линейной регрессии на исходных данных и проверил метрику R2.

In [20]:
model = LinearRegression()
model.fit(features_train_trans, target_train)
predictions = model.predict(features_test_trans)
print('R2', r2_score(target_test, predictions))

R2 0.4230772761583089


Обучил модель линейной регрессии на преобразованных данных и проверил метрику R2.

### Вывод.

В результате прктического эксперимента, мы обучили две модели линейной регресии и на практике проверили, что умножение признаков на обратимую матрицу не меняет качество линейной регрессии.